In [1]:
import pandas as pd

In [3]:
queens_data = pd.read_csv('Datasets/queensland_2020_filtered.csv')

In [8]:
queens_data.drop('Unnamed: 0', axis = 1, inplace=True)

In [10]:
queens_data.dropna(subset=['acq_time','acq_date'],inplace=True)

In [12]:
queens_data['acq_time'] = queens_data['acq_time'].astype(str).str.zfill(4)  # Ensure leading zeros for hours < 10
queens_data['acq_time'] = pd.to_datetime(queens_data['acq_time'], format='%H%M').dt.time

In [13]:
queens_data['acq_time']= pd.to_datetime(queens_data['acq_time'], format='%H:%M:%S')
queens_data['nearest_hour']=queens_data['acq_time'].dt.round('H').dt.time
queens_data['acq_time']=queens_data['acq_time'].dt.time

C:\Users\giris\AppData\Local\Temp\ipykernel_76356\404982700.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  queens_data['nearest_hour']=queens_data['acq_time'].dt.round('H').dt.time


In [3]:
#queens_data = pd.read_csv('Datasets/queens_modified.csv')
queens_data

,Unnamed: 0,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,state,nearest_hour
0,0,-20.1803,143.9641,333.8,1.1,1.0,2022-01-01,00:28:00,Terra,MODIS,73.0,6.03,306.9,18.2,D,0,Queensland,00:00:00
1,1,-20.1118,143.2919,334.7,1.2,1.1,2022-01-01,00:28:00,Terra,MODIS,71.0,6.03,309.5,16.9,D,0,Queensland,00:00:00
2,2,-20.0903,143.6725,334.9,1.1,1.1,2022-01-01,00:28:00,Terra,MODIS,50.0,6.03,312.3,15.6,D,0,Queensland,00:00:00
3,3,-20.0893,143.7275,340.8,1.1,1.1,2022-01-01,00:28:00,Terra,MODIS,80.0,6.03,306.6,32.9,D,0,Queensland,00:00:00
4,4,-20.0798,143.7292,332.9,1.1,1.1,2022-01-01,00:28:00,Terra,MODIS,57.0,6.03,307.4,20.2,D,0,Queensland,00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107364,107364,-26.6914,150.5227,317.0,2.7,1.6,2024-01-21,11:36:00,Terra,MODIS,94.0,61.03,294.3,50.7,N,0,Queensland,12:00:00
107365,107365,-26.6875,150.5158,311.6,2.7,1.6,2024-01-21,11:36:00,Terra,MODIS,83.0,61.03,294.0,31.9,N,0,Queensland,12:00:00
107366,107366,-26.6642,152.0667,300.8,2.1,1.4,2024-01-21,11:36:00,Terra,MODIS,34.0,61.03,283.8,12.4,N,0,Queensland,12:00:00
107367,107367,-26.6638,152.0610,303.6,2.1,1.4,2024-01-21,11:36:00,Terra,MODIS,56.0,61.03,282.8,17.2,N,0,Queensland,12:00:00


In [7]:
# !pip install openmeteo_requests requests_cache retry_requests

In [9]:
import openmeteo_requests

import requests_cache
from retry_requests import retry
import time

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [10]:
url = "https://archive-api.open-meteo.com/v1/archive"

In [11]:
test_data = queens_data.copy()

In [12]:
hourly_dataframe = pd.DataFrame()

In [21]:
start_index = 98780

In [22]:
count=start_index
for index, row in test_data.iloc[start_index:].iterrows():
    if(count%50==0):
        print(count,"rows done!!")
    params = {
        "latitude": row["latitude"],
        "longitude": row["longitude"],
        "start_date": row['acq_date'],
        "end_date": row['acq_date'],
        "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", 
                   "rain", "snowfall", "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", 
                   "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "et0_fao_evapotranspiration", 
                   "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_100m", "wind_direction_10m", 
                   "wind_direction_100m", "wind_gusts_10m", "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm", 
                   "soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm", "soil_moisture_0_to_7cm", 
                   "soil_moisture_7_to_28cm", "soil_moisture_28_to_100cm", "soil_moisture_100_to_255cm"],
        "timezone": "Australia/Brisbane"
    }
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    
    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
    hourly_rain = hourly.Variables(5).ValuesAsNumpy()
    hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
    hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
    hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
    hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
    hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
    hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
    hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
    hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
    hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
    hourly_et0_fao_evapotranspiration = hourly.Variables(15).ValuesAsNumpy()
    hourly_vapour_pressure_deficit = hourly.Variables(16).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(17).ValuesAsNumpy()
    hourly_wind_speed_100m = hourly.Variables(18).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()
    hourly_wind_direction_100m = hourly.Variables(20).ValuesAsNumpy()
    hourly_wind_gusts_10m = hourly.Variables(21).ValuesAsNumpy()
    hourly_soil_temperature_0_to_7cm = hourly.Variables(22).ValuesAsNumpy()
    hourly_soil_temperature_7_to_28cm = hourly.Variables(23).ValuesAsNumpy()
    hourly_soil_temperature_28_to_100cm = hourly.Variables(24).ValuesAsNumpy()
    hourly_soil_temperature_100_to_255cm = hourly.Variables(25).ValuesAsNumpy()
    hourly_soil_moisture_0_to_7cm = hourly.Variables(26).ValuesAsNumpy()
    hourly_soil_moisture_7_to_28cm = hourly.Variables(27).ValuesAsNumpy()
    hourly_soil_moisture_28_to_100cm = hourly.Variables(28).ValuesAsNumpy()
    hourly_soil_moisture_100_to_255cm = hourly.Variables(29).ValuesAsNumpy()
    
    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["latitude"] = row["latitude"]
    hourly_data["longitude"] = row["longitude"]
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["dew_point_2m"] = hourly_dew_point_2m
    hourly_data["apparent_temperature"] = hourly_apparent_temperature
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["rain"] = hourly_rain
    hourly_data["snowfall"] = hourly_snowfall
    hourly_data["snow_depth"] = hourly_snow_depth
    hourly_data["weather_code"] = hourly_weather_code
    hourly_data["pressure_msl"] = hourly_pressure_msl
    hourly_data["surface_pressure"] = hourly_surface_pressure
    hourly_data["cloud_cover"] = hourly_cloud_cover
    hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
    hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
    hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
    hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
    hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
    hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
    hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
    hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
    hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
    hourly_data["soil_temperature_7_to_28cm"] = hourly_soil_temperature_7_to_28cm
    hourly_data["soil_temperature_28_to_100cm"] = hourly_soil_temperature_28_to_100cm
    hourly_data["soil_temperature_100_to_255cm"] = hourly_soil_temperature_100_to_255cm
    hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
    hourly_data["soil_moisture_7_to_28cm"] = hourly_soil_moisture_7_to_28cm
    hourly_data["soil_moisture_28_to_100cm"] = hourly_soil_moisture_28_to_100cm
    hourly_data["soil_moisture_100_to_255cm"] = hourly_soil_moisture_100_to_255cm
    hourly_dataframe=pd.concat([hourly_dataframe,pd.DataFrame(data = hourly_data)]) 
    time.sleep(1)
    count+=1

98800 rows done!!
98850 rows done!!
98900 rows done!!
98950 rows done!!
99000 rows done!!
99050 rows done!!
99100 rows done!!
99150 rows done!!
99200 rows done!!
99250 rows done!!
99300 rows done!!
99350 rows done!!
99400 rows done!!
99450 rows done!!
99500 rows done!!
99550 rows done!!
99600 rows done!!
99650 rows done!!
99700 rows done!!
99750 rows done!!
99800 rows done!!
99850 rows done!!
99900 rows done!!
99950 rows done!!
100000 rows done!!
100050 rows done!!
100100 rows done!!
100150 rows done!!
100200 rows done!!
100250 rows done!!
100300 rows done!!
100350 rows done!!
100400 rows done!!
100450 rows done!!
100500 rows done!!
100550 rows done!!
100600 rows done!!
100650 rows done!!
100700 rows done!!
100750 rows done!!
100800 rows done!!
100850 rows done!!
100900 rows done!!
100950 rows done!!
101000 rows done!!
101050 rows done!!
101100 rows done!!
101150 rows done!!
101200 rows done!!
101250 rows done!!
101300 rows done!!
101350 rows done!!
101400 rows done!!
101450 rows done!

In [23]:
hourly_dataframe

,date,latitude,longitude,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,...,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm
0,2023-11-14 14:00:00+00:00,-17.0300,138.1816,25.650000,84.034233,22.750000,29.598408,0.0,0.0,0.0,...,349.875397,16.559999,28.750000,30.900000,29.950001,27.549999,0.184,0.163,0.168,0.185
1,2023-11-14 15:00:00+00:00,-17.0300,138.1816,25.049999,87.616188,22.850000,29.346004,0.0,0.0,0.0,...,341.564941,16.199999,28.100000,30.750000,29.950001,27.549999,0.184,0.163,0.168,0.185
2,2023-11-14 16:00:00+00:00,-17.0300,138.1816,24.549999,91.095169,23.000000,28.856995,0.0,0.0,0.0,...,326.929321,13.679999,27.600000,30.600000,29.950001,27.549999,0.184,0.163,0.168,0.185
3,2023-11-14 17:00:00+00:00,-17.0300,138.1816,23.950001,94.718010,23.049999,28.278076,0.0,0.0,0.0,...,306.430939,14.040000,27.100000,30.450001,29.950001,27.549999,0.184,0.163,0.168,0.185
4,2023-11-14 18:00:00+00:00,-17.0300,138.1816,23.600000,95.280487,22.799999,27.755463,0.0,0.0,0.0,...,292.203522,14.040000,26.700001,30.250000,29.950001,27.549999,0.184,0.163,0.168,0.185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,2024-01-21 09:00:00+00:00,-23.0698,144.1217,34.932503,35.622967,17.482500,36.706879,0.0,0.0,0.0,...,94.398621,21.599998,36.532501,32.782501,31.782499,29.332499,0.207,0.299,0.270,0.284
20,2024-01-21 10:00:00+00:00,-23.0698,144.1217,32.082500,47.071609,19.382500,34.552776,0.0,0.0,0.0,...,47.489597,9.720000,34.782501,32.882500,31.782499,29.332499,0.207,0.299,0.270,0.284
21,2024-01-21 11:00:00+00:00,-23.0698,144.1217,31.432499,48.536667,19.282499,33.816399,0.0,0.0,0.0,...,40.485935,14.759999,33.432503,32.932503,31.782499,29.332499,0.206,0.299,0.270,0.284
22,2024-01-21 12:00:00+00:00,-23.0698,144.1217,30.032499,54.915207,19.982500,32.975597,0.0,0.0,0.0,...,44.029083,11.520000,32.282501,32.882500,31.782499,29.332499,0.206,0.299,0.270,0.284


In [24]:
hourly_dataframe.to_csv('Datasets/weather_new_84511_to_END_rows.csv')

In [13]:
import subprocess

In [25]:
def VPN(action, profile=None):
    openvpn_gui_path = r"C:\Program Files\OpenVPN\bin\openvpn-gui.exe"
    valid_actions = ["connect", "disconnect", "reconnect", "status"]
    
    if action in valid_actions:
        if profile:
            command = f'"{openvpn_gui_path}" --command {action} "{profile}"'
        else:
            command = f'"{openvpn_gui_path}" --command {action}'
        subprocess.Popen(command, shell=True)
    else:
        print("Invalid action:", action)

In [26]:
def fetch_data(start_index=0):
    global hourly_dataframe  
    current_vpn = next(vpn_cycle)

    count = start_index

    for index, row in test_data.iloc[start_index:].iterrows():
        if count % 50 == 0:
            print(f"{count} rows done!!")

        try:
            # API parameters
            params = {
                "latitude": row["latitude"],
                "longitude": row["longitude"],
                "start_date": row["acq_date"],
                "end_date": row["acq_date"],
                "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", 
                           "apparent_temperature", "precipitation", "rain", "snowfall", 
                           "snow_depth", "weather_code", "pressure_msl", "surface_pressure", 
                           "cloud_cover", "cloud_cover_low", "cloud_cover_mid", 
                           "cloud_cover_high", "et0_fao_evapotranspiration", 
                           "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_100m", 
                           "wind_direction_10m", "wind_direction_100m", "wind_gusts_10m", 
                           "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm", 
                           "soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm", 
                           "soil_moisture_0_to_7cm", "soil_moisture_7_to_28cm", 
                           "soil_moisture_28_to_100cm", "soil_moisture_100_to_255cm"],
                "timezone": "Australia/Brisbane"
            }

            responses = openmeteo.weather_api(url, params=params)
            response = responses[0]
            hourly = response.Hourly()

            hourly_data = process_hourly_data(hourly, row)

            hourly_dataframe = pd.concat([hourly_dataframe, pd.DataFrame(hourly_data)], ignore_index=True)

            if count % 100 == 0:
                hourly_dataframe.to_csv("hourly_data_backup.csv", index=False)
                print("Backup saved!")

            time.sleep(1)  

        except Exception as e:
            print(f"Error at row {count}: {e}")
            print(f"Trying the next VPN profile...")

            
            VPN("disconnect", current_vpn)
            time.sleep(5)  
            current_vpn = next(vpn_cycle)
            VPN("connect", current_vpn)
            time.sleep(10)

            continue 

        count += 1

    print("Data fetching complete!")

In [27]:
def process_hourly_data(hourly, row):
    return {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()), inclusive="left"
        ),
        "latitude": row["latitude"],
        "longitude": row["longitude"],
        "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
        "relative_humidity_2m": hourly.Variables(1).ValuesAsNumpy(),
        "dew_point_2m": hourly.Variables(2).ValuesAsNumpy(),
        "apparent_temperature": hourly.Variables(3).ValuesAsNumpy(),
        "precipitation": hourly.Variables(4).ValuesAsNumpy(),
        "rain": hourly.Variables(5).ValuesAsNumpy(),
        "snowfall": hourly.Variables(6).ValuesAsNumpy(),
        "snow_depth": hourly.Variables(7).ValuesAsNumpy(),
        "weather_code": hourly.Variables(8).ValuesAsNumpy(),
        "pressure_msl": hourly.Variables(9).ValuesAsNumpy(),
        "surface_pressure": hourly.Variables(10).ValuesAsNumpy(),
        "cloud_cover": hourly.Variables(11).ValuesAsNumpy(),
        "cloud_cover_low": hourly.Variables(12).ValuesAsNumpy(),
        "cloud_cover_mid": hourly.Variables(13).ValuesAsNumpy(),
        "cloud_cover_high": hourly.Variables(14).ValuesAsNumpy(),
        "et0_fao_evapotranspiration": hourly.Variables(15).ValuesAsNumpy(),
        "vapour_pressure_deficit": hourly.Variables(16).ValuesAsNumpy(),
        "wind_speed_10m": hourly.Variables(17).ValuesAsNumpy(),
        "wind_speed_100m": hourly.Variables(18).ValuesAsNumpy(),
        "wind_direction_10m": hourly.Variables(19).ValuesAsNumpy(),
        "wind_direction_100m": hourly.Variables(20).ValuesAsNumpy(),
        "wind_gusts_10m": hourly.Variables(21).ValuesAsNumpy(),
        "soil_temperature_0_to_7cm": hourly.Variables(22).ValuesAsNumpy(),
        "soil_temperature_7_to_28cm": hourly.Variables(23).ValuesAsNumpy(),
        "soil_temperature_28_to_100cm": hourly.Variables(24).ValuesAsNumpy(),
        "soil_temperature_100_to_255cm": hourly.Variables(25).ValuesAsNumpy(),
        "soil_moisture_0_to_7cm": hourly.Variables(26).ValuesAsNumpy(),
        "soil_moisture_7_to_28cm": hourly.Variables(27).ValuesAsNumpy(),
        "soil_moisture_28_to_100cm": hourly.Variables(28).ValuesAsNumpy(),
        "soil_moisture_100_to_255cm": hourly.Variables(29).ValuesAsNumpy(),
    }

In [28]:
fetch_data(start_index=51211)

Error at row 51211: {'error': True, 'reason': 'Daily API request limit exceeded. Please try again tomorrow.'}
Trying the next VPN profile...
Error at row 51211: {'reason': 'Daily API request limit exceeded. Please try again tomorrow.', 'error': True}
Trying the next VPN profile...
Error at row 51211: {'error': True, 'reason': 'Daily API request limit exceeded. Please try again tomorrow.'}
Trying the next VPN profile...
Error at row 51211: {'error': True, 'reason': 'Daily API request limit exceeded. Please try again tomorrow.'}
Trying the next VPN profile...
Error at row 51211: {'error': True, 'reason': 'Daily API request limit exceeded. Please try again tomorrow.'}
Trying the next VPN profile...
Error at row 51211: {'error': True, 'reason': 'Daily API request limit exceeded. Please try again tomorrow.'}
Trying the next VPN profile...
Error at row 51211: {'reason': 'Daily API request limit exceeded. Please try again tomorrow.', 'error': True}
Trying the next VPN profile...
Error at row 

KeyboardInterrupt: 

In [24]:
hourly_dataframe.to_csv('Datasets/weather_new_44677_to_51211_rows.csv')

In [17]:
from itertools import cycle

In [18]:
vpn_profiles = [
    "jp-free-266024.protonvpn.udp.ovpn",
    "jp-free-244008.protonvpn.udp.ovpn",
    "us-free-543039.protonvpn.udp.ovpn",
    "us-free-101032.protonvpn.udp.ovpn",
    "us-free-102069.protonvpn.udp.ovpn",
    "us-free-128091.protonvpn.udp.ovpn"
]
vpn_cycle = cycle(vpn_profiles)